In [1]:
from deep_traffic_generation.tcvae_pairs_disent import TCVAE_Pairs_disent
from deep_traffic_generation.VAE_Generation import PairsVAE
from traffic.algorithms.generation import Generation
from deep_traffic_generation.core.datasets import TrafficDatasetPairsRandom
from traffic.core import Traffic

from sklearn.preprocessing import MinMaxScaler
import openturns as ot
import torch
import pandas as pd
import numpy as np
from os import walk

In [2]:
dataset = TrafficDatasetPairsRandom.from_file(
    ("../deep_traffic_generation/data/training_datasets/to_LSZH_16_50_bb.pkl", "../deep_traffic_generation/data/training_datasets/ga_LSZH_14_50_bb.pkl"),
    features=["track", "groundspeed", "altitude", "timedelta"],
    n_samples = 10000,
    scaler=MinMaxScaler(feature_range=(-1,1)),
    shape="image",
    info_params={"features": ["latitude", "longitude"], "index": -1},
)

path = "../deep_traffic_generation/lightning_logs/tcvae_pairs_disent/version_22/"

t = PairsVAE(X = dataset, vae_type="TCVAEPairs_disent", sim_type = "generation")
t.load(path, dataset.parameters)
gen_vae = Generation(generation=t, features = t.VAE.hparams.features, scaler=dataset.scaler) 

# MC by hand

In [31]:
n_gen = 1000

Z = t.latent_space(0)

p_z = t.VAE.lsr.get_prior()
z_gen = p_z.sample(torch.Size([n_gen])).squeeze(1)
if len(z_gen.shape) == 3:
    z_gen = z_gen.squeeze(2)
gen = t.decode(z_gen)

gen_traf_to = gen_vae.build_traffic(gen[:,:200], coordinates = dict(latitude =  47.44464, longitude = 8.55732), forward=True)
gen_traf_to = gen_traf_to.assign(flight_id=lambda x: x.flight_id + "_to", inplace=True)
gen_traf_to = gen_traf_to.resample("1s").eval()

gen_traf_ga = gen_vae.build_traffic(gen[:,200:], coordinates = dict(latitude = 47.500086, longitude = 8.51149), forward=True)
gen_traf_ga = gen_traf_ga.assign(flight_id=lambda x: x.flight_id + "_ga", inplace=True)
gen_traf_ga = gen_traf_ga.resample("1s").eval()



In [54]:
from multiprocessing import Pool
import os

def reassign_timestamp(iter):
    t, ga = iter
    nb = int(ga.flight_id.split("_")[1])
    
    diff = ga.start - t.start

    ga.data = ga.data.assign(
        timestamp = ga.data.timestamp - diff + pd.to_timedelta(nb*15, unit="Min")  
        )
    
    t.data = t.data.assign(
        timestamp = t.data.timestamp + pd.to_timedelta(nb*15, unit="Min")  
        )
    
    return t+ga
    
with Pool(processes=os.cpu_count()-10) as p: 
    result = p.map(reassign_timestamp, zip(gen_traf_to, gen_traf_ga))
    p.close()
    p.join()
results = sum(result)    

In [ ]:
# test = Traffic.from_file("../deep_traffic_generation/gen_crude_mc/gen_traf_gato.parquet")

# MC with OT

In [3]:
Z = t.latent_space(0)
p_z = t.VAE.lsr.get_prior()

marginals = []
for i in range(Z.shape[1]):
    collDist = [ot.Normal(mu.item(), sigma.item()) for mu, sigma in zip(p_z.base_dist.component_distribution.base_dist.loc.squeeze(2)[i], p_z.base_dist.component_distribution.base_dist.scale.squeeze(2)[i])]
    weights = p_z.base_dist.mixture_distribution.probs[i].detach().numpy()
    mixt = ot.Mixture(collDist, weights)
    marginals.append(mixt)
prior = ot.ComposedDistribution(marginals)

In [4]:
event_inputs = []

def limit_state(z):
    """limit_state is the black box function for the sumbset samling. 

    inputs:
        - z = a latent space vector from N(0,1)
        
    outputs:
        - limit_state(X) = closest distance between the two trajectories of the pair
    """
    diam = 55
    
    z = np.array(z).reshape(1,-1)
    z = torch.Tensor(z)
    
    #Decode latent representation into a pair of trajectories
    decoded = t.decode(z)
    to = gen_vae.build_traffic(decoded[:,:200], coordinates = dict(latitude =  47.44464, longitude = 8.55732), forward=True).iterate_lazy().resample("1s").eval()
    to = to.assign(flight_id=lambda x: x.flight_id + "_to", inplace=True)
    ga = gen_vae.build_traffic(decoded[:,200:], coordinates = dict(latitude = 47.500086, longitude = 8.51149), forward=True).iterate_lazy().resample("1s").eval() 
    ga = ga.assign(flight_id=lambda x: x.flight_id + "_ga", inplace=True)
    
    # Calulate distance between the two trajectories
    dist = to[0].distance(ga[0])
    dist["3d_distance"] = dist.apply(lambda x: ((x.lateral*1852)**2 + (x.vertical*0.3048)**2)**0.5 - diam, axis=1) #distance between two spheres in m
    min_dist = dist["3d_distance"].min()
    
    #keep inputs that are in the failure domain
    if min_dist < 0:
        event_inputs.append(z)
    
    return [min_dist]

In [11]:
X = ot.RandomVector(prior)
g = ot.PythonFunction(10, 1, limit_state)
f = ot.MemoizeFunction(g)
Y = ot.CompositeRandomVector(f, X)

In [12]:
myEvent = ot.ThresholdEvent(Y, ot.LessOrEqual(), 0.0) 
experiment = ot.MonteCarloExperiment()
algo = ot.ProbabilitySimulationAlgorithm(myEvent, experiment)
algo.setMaximumOuterSampling(int(1e2))
algo.run()

In [13]:
result = algo.getResult()
probability = result.getProbabilityEstimate()
print("Pf=", probability)

Pf= 0.0


In [8]:
import pickle as pkl
with open("mc_failures", "wb") as fp:
    pkl.dump(event_inputs, fp)

In [14]:
p = 3*1e-4
n = 50000
cv = np.sqrt((1-p)/(n*p))
cv

0.25816015700852574